In [1]:
import streamlit as st
import base64
from datetime import datetime
import io
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import warnings
import time
import numpy as np
today_date = datetime.today().strftime('%d-%m-%Y')
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [27]:
url = "https://www.screener.in/company/ROLEXRINGS/"
response = requests.get(url)

soup = bs(response.content, "html.parser")

In [28]:
quarterly_shp_section = soup.find('div', {'id': 'quarterly-shp'})

shareholding_table = quarterly_shp_section.find('table', {'class': 'data-table'})
shareholding_data = []
for row in shareholding_table.find_all('tr'):
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    shareholding_data.append(cols)

df = pd.DataFrame(shareholding_data)
df = df.transpose()
df = df.drop(columns=[0])
df.columns = df.iloc[0]
df = df.iloc[1:]

In [31]:
# Define a dictionary to map the starting strings to the new names
rename_dict = {
    'Pro': 'Promoter',
    'FII': 'FII',
    'DII': 'DII',
    'Gov': 'Government',
    'Oth': 'Others',
    'Pub': 'Public'
}

# Function to apply the renaming logic
def rename_columns(col_name):
    for key, value in rename_dict.items():
        if col_name.startswith(key):
            return value
    return col_name  # Return the original name if no match is found

# Rename the columns using the function
df.columns = [rename_columns(col) for col in df.columns]


I want to convert their names 

like in this 
print(df.columns)
Index(['Promoters +', 'FIIs +', 'DIIs +', 'Public +', 'No. of Shareholders'], dtype='object', name=0)

if column name is starting with "Pro" then it's "Promoter"
if column name is starting with "FII" then it's "FII"
if column name is starting with "DII" then it's "DII"
if column name is starting with "Gov" then it's "Government"
if column name is starting with "Oth" then it's "Others"



In [32]:
df

,Promoter,FII,DII,Public,No. of Shareholders
1,57.64%,4.06%,17.91%,20.40%,"79,708"
2,57.64%,3.50%,19.37%,19.49%,"60,169"
3,57.64%,1.92%,20.37%,20.07%,"51,555"
4,57.64%,1.71%,22.39%,18.26%,"46,042"
5,57.64%,3.18%,32.69%,6.48%,"46,752"
6,57.64%,3.93%,31.93%,6.51%,"46,794"
7,57.64%,4.06%,31.76%,6.53%,"45,757"
8,55.25%,5.01%,32.47%,7.28%,"48,497"
9,55.25%,5.60%,32.09%,7.06%,"49,141"


In [33]:
df['Promoter'] = df['Promoter'].str.replace('%', '').astype(float)
df['FII'] = df['FII'].str.replace('%', '').astype(float)
df['DII'] = df['DII'].str.replace('%', '').astype(float)
df['Public'] = df['Public'].str.replace('%', '').astype(float)

In [34]:
df

,Promoter,FII,DII,Public,No. of Shareholders
1,57.64,4.06,17.91,20.40,"79,708"
2,57.64,3.50,19.37,19.49,"60,169"
3,57.64,1.92,20.37,20.07,"51,555"
4,57.64,1.71,22.39,18.26,"46,042"
5,57.64,3.18,32.69,6.48,"46,752"
6,57.64,3.93,31.93,6.51,"46,794"
7,57.64,4.06,31.76,6.53,"45,757"
8,55.25,5.01,32.47,7.28,"48,497"
9,55.25,5.60,32.09,7.06,"49,141"


In [35]:
df['Public % Change'] = df['Public'].pct_change() * 100
df['FII % Change'] = df['FII'].pct_change() * 100
df['DII % Change'] = df['DII'].pct_change() * 100
df['Promoter % Change'] = df['Promoter'].pct_change() * 100

# The first row will have NaN values because there is no previous row to compare with
# You may want to fill NaN values with 0 or leave them as is, depending on your requirement
df.fillna(0, inplace=True)

In [36]:
df

,Promoter,FII,DII,Public,No. of Shareholders,Public % Change,FII % Change,DII % Change,Promoter % Change
1,57.64,4.06,17.91,20.40,"79,708",0.000000,0.000000,0.000000,0.000000
2,57.64,3.50,19.37,19.49,"60,169",-4.460784,-13.793103,8.151870,0.000000
3,57.64,1.92,20.37,20.07,"51,555",2.975885,-45.142857,5.162623,0.000000
4,57.64,1.71,22.39,18.26,"46,042",-9.018435,-10.937500,9.916544,0.000000
5,57.64,3.18,32.69,6.48,"46,752",-64.512596,85.964912,46.002680,0.000000
6,57.64,3.93,31.93,6.51,"46,794",0.462963,23.584906,-2.324870,0.000000
7,57.64,4.06,31.76,6.53,"45,757",0.307220,3.307888,-0.532415,0.000000
8,55.25,5.01,32.47,7.28,"48,497",11.485452,23.399015,2.235516,-4.146426
9,55.25,5.60,32.09,7.06,"49,141",-3.021978,11.776447,-1.170311,0.000000


In [ ]:
df[['Public % Change', 'FII % Change', 'DII % Change', 'Promoter % Change']]

In [38]:
df["Score"] = (df['Promoter % Change']+df['DII % Change']+df['FII % Change'])-df['Public % Change'] 

In [43]:
import plotly as plt

In [44]:
import plotly.express as px

In [45]:
fig = px.bar(df, x=df.index, y='Score', title='Score Bar Graph')

In [47]:
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed